In [ ]:
%pip install llama-index-llms-openai
!pip install llama-index
#import nest_asyncio

#nest_asyncio.apply()
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [2]:
from llama_index.core import SimpleDirectoryReader

# documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()



In [3]:
from llama_index.core.node_parser import SentenceSplitter

nodes = SentenceSplitter().get_nodes_from_documents(documents)


In [ ]:
nodes

In [4]:
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)


In [ ]:
len(docstore.docs)


In [6]:
#os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")
from llama_index.core import Settings
from llama_index.llms.ollama import  Ollama
Settings.llm = Ollama(model='llama3.2:latest', base_url='http://localhost:11434',temperature=0.1)



In [7]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",  # Replace with your desired model
    base_url="http://localhost:11434",  # Ensure Ollama is running at this endpoint
    ollama_additional_kwargs={"mirostat": 0} #Mirostat is a technique for controlling perplexity and balancing the text generation process in large language models (LLMs).
) 

In [8]:
from llama_index.core import StorageContext, SummaryIndex, VectorStoreIndex, SimpleKeywordTableIndex

storage_context = StorageContext.from_defaults(docstore=docstore)
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=ollama_embedding)
keyword_table_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)


In [ ]:
query_engine = summary_index.as_query_engine()
response = query_engine.query("What this document is about?")
print(response)




In [ ]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)


In [ ]:
query_engine = keyword_table_index.as_query_engine()
response = query_engine.query("Which company annual or quarterly report is this?")
print(response)

In [ ]:
len(docstore.docs)

In [13]:
from llama_index.core import StorageContext

# Assume `docstore` is your DocumentStore
# storage_context = StorageContext.from_defaults(docstore=docstore)

# Persist the StorageContext to disk
storage_context.persist("storage_context/")

In [ ]:
from llama_index.core.schema import Document, TextNode
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SentenceSplitter

# Initialize the DocumentStore
docstore = SimpleDocumentStore()

# Example documents to add
documents = [
    Document(text="This is the first document to persist store."),
    Document(text="This is the second document to persist store."),
]

# Parse the documents into nodes
nodes = SentenceSplitter().get_nodes_from_documents(documents)

# Add nodes to the DocumentStore
docstore.add_documents(nodes)

# Persist the DocumentStore to disk
persist_path = "persist_store"
docstore.persist(persist_path)

print(f"DocumentStore persisted to {persist_path}")

